## Ray

#### Was ist Ray?

Den ganzen Kurs über haben wir das Ray-Paket benutzt:

In [1]:
import ray.rllib

![](img/ray-logo.png)

Was ist Ray? Aus den [docs](https://docs.ray.io/en/latest/):

&gt; Ray ist ein universell einsetzbares, verteiltes Computing-Framework.

Ray ist auch:

- Ein [aktives Open-Source-Projekt](https://github.com/ray-project/ray) mit über 20k Sternen auf GitHub 🤩
- Unterstützt von dem Einhorn-Startup [Anyscale](https://www.anyscale.com/), das diesen Kurs produziert hat 🦄

Anmerkungen:

Aber zurück zum verteilten Rechnen.

#### Was ist verteiltes Rechnen?

_Verteiltes Rechnen_ ist das Rechnen mit mehreren Rechnern (Knoten), die über ein Netzwerk verteilt sind.

![](img/supercomputer.png)

Vorteile:

- Massiv verbesserte Fähigkeiten

Nachteile/Herausforderungen:

- Synchronisation
- Misserfolg
- ...

#### Ray macht verteiltes Rechnen einfach

- Das Ziel von Ray ist es, verteiltes Rechnen einfach und zugänglich zu machen.
- Ray übernimmt die meisten Herausforderungen für die Nutzer.
- RLlib, tune und die anderen Unterpakete wurden auf der Grundlage von Ray entwickelt.
- Das bedeutet, dass _RLlib und tune automatisch über verteilte Fähigkeiten verfügen._

Anmerkungen:

Überraschung! RLlib ist einfach zu benutzen und implementiert viele moderne RL-Algorithmen, aber es hat noch einen weiteren Vorteil, den wir bisher nicht erwähnt haben: natürliche verteilte Berechnungsmöglichkeiten. Damit ist sie konkurrierenden Paketen weit voraus, wenn es darum geht, die Berechnungen zu verteilen.

#### RLlib, verteilt

- In diesem Kurs haben wir viele Algorithmus-Konfigurationen erstellt.
- Aber es gibt einige Parameter, die wir noch nicht verwendet haben:

In [2]:
from ray.rllib.algorithms.ppo import PPOConfig

In [3]:
ppo_config = (
    PPOConfig()
    .framework("torch")
    .rollouts(num_rollout_workers=4, num_envs_per_worker=2)
    .resources(num_gpus=0)
)

Du kannst mehr über die Angabe von Ressourcen [hier](https://docs.ray.io/en/master/rllib/rllib-training.html#specifying-resources) und über Skalierung [hier](https://docs.ray.io/en/master/rllib/rllib-training.html#scaling-guide) lesen.

Aber... was ist ein "Rollout Worker"?

#### Rollout Arbeiter

- Rollout Worker sammeln parallel Daten aus der Environment (Simulator).
- Für die meisten Simulatorumgebungen kann man die Environment in einem Cluster replizieren.
- So kannst du viel schneller Daten sammeln und Engpässe beim Training vermeiden.
- Egal, mit welchem Cluster Ray im Backend verbunden ist, `num_rollout_workers=4` funktioniert nahtlos.

Anmerkungen:

Wenn du beim überwachten Lernen wartest, weißt du, dass du wahrscheinlich darauf wartest, dass das Modell trainiert wird. Im RL könnte der Engpass die Datensammlung oder die Modellaktualisierung sein. Die Möglichkeit, Rollouts zu parallelisieren, lindert den Engpass bei der Datenerfassung 

#### Ray tune, revisited

- Vergiss nicht, dass das Abstimmen von Hyperparametern wie die Rastersuche auch leicht verteilt werden kann.
- Glücklicherweise ist `tune` auch Teil von Ray und kümmert sich, wie RLlib, für dich darum! 
- Wie du siehst, ist Ray + tune + RLlib eine ziemlich mächtige Kombination.

#### Treiber

In allen unseren Konfigurationen haben wir

```python
create_env_on_driver = True
```

Das bedeutet, dass wir die Environmentsvariable auf denselben "Treiber"-Prozess setzen, der das Training ausführt.

#### Zusammenfassung

- Ray ist unglaublich mächtig, und wir haben nur die Spitze des Eisbergs angekratzt.
- Einige andere Ressourcen:
  - [ray.io](https://www.ray.io/)
  - [Learning Ray](https://www.oreilly.com/library/view/learning-ray/9781098117214/) (Buch)

#### Lass uns das Gelernte anwenden!

## Was ist Ray?
<!-- multiple choice -->

Was ist Ray?

- [ ] Das Unternehmen, das RLlib entwickelt. | Du denkst vielleicht an Anyscale, die Firma hinter Ray!
- [Ein Unterpaket der RLlib, das sich mit verteiltem Rechnen beschäftigt.
- [Ein allgemeines Paket, das die RLlib enthält, die sich mit verteiltem Rechnen befasst.
- [x] Ein Algorithmus für Reinforcement Learning 

## RLlib verteilen
<!-- multiple choice -->

Wie nutzt RLlib in erster Linie die Möglichkeiten des verteilten Rechnens?

- [x] Verteilte Rollout-Worker generieren Daten von Environmentsklonen, die in den Lernalgorithmus eingespeist werden 
- [Das Training des neuronalen Netzwerks wird auf mehrere Knotenpunkte verteilt.
- [Jeder Knoten verfügt über ein separates neuronales Netzwerk, das unabhängig auf seinem eigenen Knoten trainiert wird.

## Experimentieren mit Rollout-Arbeitern
<!-- coding exercise -->

Der folgende Code erstellt zwei Instanzen des PPO-Algorithmus: eine, die zwei Rollout-Worker mit zwei Environmenten pro Worker verwenden soll, und eine, die nur einen Rollout-Worker mit einer Environment pro Worker verwendet. Anschließend wird die Zeit ausgedruckt, die für das Training der beiden Instanzen für 5 Iterationen benötigt wurde. Vervollständige den Code, führe ihn aus und vergleiche dann die Zeiten 

Hinweis: Dieses Experiment funktioniert _normalerweise_. Allerdings läuft dieser Code auf einem Server, der möglicherweise von mehreren Lernenden gleichzeitig genutzt wird. Außerdem ist dieser Server kein richtiger Cluster, sodass die Parallelisierung auf einem Rechner nur funktioniert, wenn mehrere CPU-Kerne zur Verfügung stehen.

In [4]:
# EXERCISE

from ray.rllib.algorithms.ppo import PPOConfig
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config_many = (
    PPOConfig()
    .framework("torch")
    .____(____)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_config_single = (
    PPOConfig()
    .framework("torch")
    .____(num_rollout_workers=1, num_envs_per_worker=1)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_many = ppo_config_many.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_many.train()
print(f"Elapsed time with 2 workers, 2 envs each: {time.time()-t:.1f}s.")
ppo_many.stop()

ppo_single = ppo_config_single.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_single.train()
print(f"Elapsed time with 1 worker, 1 env: {time.time()-t:.1f}s.")
ppo_single.stop()

AttributeError: 'PPOConfig' object has no attribute '____'

In [5]:
# SOLUTION

from ray.rllib.algorithms.ppo import PPOConfig
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

ppo_config_many = (
    PPOConfig()
    .framework("torch")
    .rollouts(num_rollout_workers=2, num_envs_per_worker=2)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_config_single = (
    PPOConfig()
    .framework("torch")
    .rollouts(num_rollout_workers=1, num_envs_per_worker=1)
    .training(model={"fcnet_hiddens" : [32,32]})
)

ppo_many = ppo_config_many.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_many.train()
print(f"Elapsed time with 2 workers, 2 envs each: {time.time()-t:.1f}s.")
ppo_many.stop()

ppo_single = ppo_config_single.build(env="FrozenLake-v1")
t = time.time()
for i in range(5):
    ppo_single.train()
print(f"Elapsed time with 1 worker, 1 env: {time.time()-t:.1f}s.")
ppo_single.stop()

2022-08-27 17:02:42,927	INFO worker.py:1490 -- Started a local Ray instance.
(RolloutWorker pid=70484) 2022-08-27 17:02:47,263	WARNING env.py:154 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.
(RolloutWorker pid=70484) 2022-08-27 17:02:47,536	WARNING deprecation.py:47 -- DeprecationWarning: `concat_samples` has been deprecated. Use `concat_samples() from rllib.policy.sample_batch` instead. This will raise an error in the future!


Elapsed time with 2 workers, 2 envs each: 11.3s.


(RolloutWorker pid=70561) 2022-08-27 17:03:00,941	WARNING env.py:154 -- Your env reset() method appears to take 'seed' or 'return_info' arguments. Note that these are not yet supported in RLlib. Seeding will take place using 'env.seed()' and the info dict will not be returned from reset.


Elapsed time with 1 worker, 1 env: 18.0s.
